Author: Lai Khee Jiunn
  
  

Source: 

    [1] http://www.bursamalaysia.com/market/listed-companies/list-of-companies/main-market/
    [2] https://klse.i3investor.com/
    [3] https://www.klsescreener.com/
  
  

Reference:

    [1] K. M. Ho, "How to make money from your stock investment even in a falling market", 3rd ed., Malaysia, Kanyin Publications Sdn. Bhd., 2017.

Note: The entire process in this jupyter notebook may take up to 2.5 hours depending on your internet speed

In [1]:
import numpy as np
import pandas as pd
import re, time, os
from datetime import datetime, date
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
# from time import time
from tqdm import tqdm
from glob import glob
chromedriver = 'D://Workspace/selenium_driver/bin/chromedriver'
out_dir = 'resutt_2020-04-04/'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [2]:
# stock_code = pd.read_csv(out_dir+'stock_code_table.csv')
# data1 = pd.read_csv(out_dir+'klse_screener_52week_price.csv')
# data2 = pd.read_csv(out_dir+'klse_morningstar_merge_split.csv', parse_dates=['split_date'])
# data2['code'] = data2['code'].astype(str).apply(lambda x: x.zfill(4))
# data3 = pd.read_csv(out_dir+'klse_investor_10yr.csv')
# data3['code'] = data3['code'].astype(str).apply(lambda x: x.zfill(4))
# data4 = pd.read_csv(out_dir+'klse_stock_stage_4.csv')
# data4['code'] = data4['code'].astype(str).apply(lambda x: x.zfill(4))
# data5 = pd.read_csv(out_dir+'klse_stock_intrinsic.csv')
# data5['code'] = data5['code'].astype(str).apply(lambda x: x.zfill(4))
# data6 = pd.read_csv(out_dir+'klse_stock_dividend_yield.csv')
# data6['code'] = data6['code'].astype(str).apply(lambda x: x.zfill(4))

### Get all listed company names and stock codes from Bursa Malaysia

In [3]:
options = webdriver.ChromeOptions()
options.add_argument("--incognito")
driver = webdriver.Chrome(chromedriver, options=options)
driver.switch_to.window(driver.current_window_handle)
driver.maximize_window()
driver.get('http://www.bursamalaysia.com/market/listed-companies/list-of-companies/main-market/')
driver.find_element_by_class_name('cc-btn').click()
time.sleep(2)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight + 200);")
time.sleep(2)
driver.find_element_by_class_name('custom-select').click()
driver.find_element_by_class_name('custom-select').find_elements_by_tag_name('option')[-1].click()
time.sleep(2)
x = driver.find_elements_by_class_name('listed-company-table')[-1]\
          .find_element_by_tag_name('tbody')\
          .find_elements_by_class_name('company-announcement-link')

code, name = [],[]
for i in tqdm(x):
    code.append(i.get_attribute('href').split('=')[-1])
    name.append(i.get_attribute('innerText').strip())
driver.quit()

100%|████████████████████████████████████████████████████████████████████████████| 789/789 [00:07<00:00, 100.82it/s]


In [4]:
stock_code = pd.DataFrame({'code':code,'name':name})
idx1 = stock_code[stock_code.name == 'KLCC REAL ESTATE INVESTMENT TRUST'].index
idx2 = stock_code[stock_code.name == 'KLCC PROPERTY HOLDINGS BERHAD'].index
stock_code.loc[idx1, 'code'] = '5235SS'
stock_code.drop(index=idx2,inplace=True)
stock_code.reset_index(drop=True)
print(stock_code.shape)
stock_code.to_csv(out_dir+'stock_code_table.csv', index=False)
stock_code.head(1)

(789, 2)


,code,name
0,5250,7-ELEVEN MALAYSIA HOLDINGS BERHAD


### Stage 1: Get 52w high and low price

In [5]:
# Get stock prices in 52 weeks range from KLSE Screener, which will be used in Stage 4
start_time = time.time()
options = webdriver.ChromeOptions()
options.add_argument("--incognito")
driver = webdriver.Chrome(chromedriver, options=options)
driver.switch_to.window(driver.current_window_handle)
driver.maximize_window()
error1, result1 = [],[]
for i in tqdm(stock_code.code):
    try:
        driver.get('https://www.klsescreener.com/v2/stocks/view/'+i)
        cat = driver.find_element_by_css_selector('div:nth-child(3) > div.col-xl-4 > span:nth-child(2)')\
                    .get_attribute('innerText')\
                    .split(' - ')[0]
        price = driver.find_element_by_id('price')\
                      .get_attribute('innerText')
        week52 = driver.find_element_by_css_selector('div:nth-child(3) > div.col-xl-4 > div > \
                                     table:nth-child(1) > tbody > tr:nth-child(7) > td:nth-child(2)')\
                       .get_attribute('innerText')\
                       .split(' - ')
        cap = driver.find_element_by_css_selector('div:nth-child(3) > div.col-xl-4 > div > \
                                     table:nth-child(1) > tbody > tr:nth-child(17) > td:nth-child(2)')\
                    .get_attribute('innerText')
        nta = driver.find_element_by_id('quarter_reports')\
                    .find_element_by_css_selector('table > tbody > tr:nth-child(1) > td:nth-child(3)')\
                    .get_attribute('innerText')
        result1.append([str(i), str(cat), float(price), float(week52[0]), float(week52[-1]), cap, float(nta)])
    except:
        error1.append(str(i))
driver.quit()
print('Time taken: ',time.time()-start_time)
data1 = pd.DataFrame(result1, columns=['code','category','current_price','52w low','52w high','market cap', 'NTA'])
print(data1.shape)
data1.to_csv(out_dir+'klse_screener_52week_price.csv', index=False)
display(data1.head(1))

100%|█████████████████████████████████████████████████████████████████████████████| 789/789 [22:05<00:00,  1.68s/it]


Time taken:  1337.2405745983124
(788, 7)


,code,category,current_price,52w low,52w high,market cap,NTA
0,5250,Consumer Products & Services,1.36,1.21,1.56,1,0.0893


### Stage 2: Get Share Merges & Splits

In [3]:
# Get stock prices in 52 weeks range from KLSE Screener, which will be used in Stage 4
start_time = time.time()
options = webdriver.ChromeOptions()
options.add_argument("--incognito")
driver = webdriver.Chrome(chromedriver, options=options)
driver.switch_to.window(driver.current_window_handle)
driver.maximize_window()
error2, result2 = [],[]
for i in tqdm(stock_code.code):
    try:
        driver.get('http://performance.morningstar.com/stock/performance-return.action?p=dividend_split_page&t='+str(i)+'&region=mys&culture=en-US')
#         splits = driver.find_element_by_css_selector('#div_split_history > table:nth-child(3) > tbody:nth-child(3)') # For Firefox
        splits = driver.find_element_by_css_selector('#splittable > tbody') # For Chrome
        if splits.text:
            splits_2 = [x.split(' ') for x in splits.text.split('\n')]
            for sp in splits_2:
                result2.append([str(i)] + sp)
    except:
        error2.append(i)
driver.quit()
print('Time taken: ',time.time()-start_time)
data2 = pd.DataFrame(result2, columns=['code','split_date','ratio'])
print(data2.shape)
data2.to_csv(out_dir+'klse_morningstar_merge_split.csv', index=False)
display(data2.head(1))

### Stage 3: Get past 10 years data

In [ ]:
# last time 5293 dont have record, check again
# balance = balance[~balance.code.isin(['5293'])]
# balance.shape

In [6]:
# Get the last 10 years of financial data from klse.i3investor.com
start_time = time.time()
driver = webdriver.Chrome(chromedriver)
driver.switch_to.window(driver.current_window_handle)
driver.maximize_window()
data3, error3 = pd.DataFrame(), []
for i in tqdm(data1.code):
    try:
        driver.get('https://klse.i3investor.com/servlets/stk/fin/'+str(i)+'.jsp?type=last10fy')
        cat = driver.find_element_by_class_name('boarAndSector').get_attribute('innerText').split(' : ')[-1]
        x = driver.find_elements_by_tag_name('table')[13].find_elements_by_tag_name('tr')
        if len(x) != 119:
            x = driver.find_elements_by_tag_name('table')[14].find_elements_by_tag_name('tr')
        year_end   = [str(i), cat, 'Year End'] + [x.get_attribute('innerText') for x in x[1].find_elements_by_tag_name('th')[3:-1]]
        net_profit = [str(i), cat, 'NP'] + [x.get_attribute('innerText') for x in x[15].find_elements_by_tag_name('td')[3:-1]]
        div        = [str(i), cat, 'Div'] + [x.get_attribute('innerText') for x in x[31].find_elements_by_tag_name('td')[3:-1]]
        div_pay    = [str(i), cat, 'Div Payout'] + [x.get_attribute('innerText') for x in x[34].find_elements_by_tag_name('td')[3:-1]]
    #     net_worth  = [str(i), cat, 'Net Worth'] + [x.get_attribute('innerText') for x in x[39].find_elements_by_tag_name('td')[3:-1]]
        nosh       = [str(i), cat, 'NOSH'] + [x.get_attribute('innerText') for x in x[42].find_elements_by_tag_name('td')[3:-1]]
        np_margin  = [str(i), cat, 'NP Margin'] + [x.get_attribute('innerText') for x in x[47].find_elements_by_tag_name('td')[3:-1]]
        roe        = [str(i), cat, 'ROE'] + [x.get_attribute('innerText') for x in x[50].find_elements_by_tag_name('td')[3:-1]]
        eps        = [str(i), cat, 'EPS'] + [x.get_attribute('innerText') for x in x[58].find_elements_by_tag_name('td')[3:-1]]
        eps_gr     = [str(i), cat, 'EPS GR'] + [x.get_attribute('innerText') for x in x[59].find_elements_by_tag_name('td')[3:-1]]
        dps        = [str(i), cat, 'DPS'] + [x.get_attribute('innerText') for x in x[61].find_elements_by_tag_name('td')[3:-1]]
        price      = [str(i), cat, 'Price'] + [x.get_attribute('innerText') for x in x[84].find_elements_by_tag_name('td')[3:-1]]
        pe         = [str(i), cat, 'P/E'] + [x.get_attribute('innerText') for x in x[88].find_elements_by_tag_name('td')[3:-1]]
        dy         = [str(i), cat, 'DY'] + [x.get_attribute('innerText') for x in x[94].find_elements_by_tag_name('td')[3:-1]]
        data3 = pd.concat([data3, pd.DataFrame([year_end, net_profit, div, div_pay, nosh, np_margin, roe, eps, eps_gr, dps, price, pe, dy])])
    except Exception as e:
        error3.append([i, type(e).__name__])
driver.quit()
print('Time taken: ',time.time()-start_time)
data3.columns = ['code','category','detail','Y1','Y2','Y3','Y4','Y5','Y6','Y7','Y8','Y9','Y10']
data3['code'] = data3['code'].astype(str)
print(data3.shape)
data3.to_csv(out_dir+'klse_investor_10yr.csv', index=False)
data3.head(1)

100%|█████████████████████████████████████████████████████████████████████████████| 788/788 [40:26<00:00,  3.08s/it]


Time taken:  2438.019079208374
(10231, 13)


,code,category,detail,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10
0,5250,Consumer,Year End,31/12/19,31/12/18,31/12/17,31/12/16,31/12/15,31/12/14,31/12/13,NaN,NaN,NaN


In [7]:
data3.code.nunique()

787

### Stage 4: Process each stock

In [8]:
error4, error4a, result4, symbol= [], [], [], ['  -  ', '-', '- %', '\xa0-\xa0', '\xa0-\xa0 %', '-\xa0 %', 'nan']
for i in tqdm(data3.code.unique()):    
    stock = data3[data3.code == str(i).zfill(4)].fillna('  -  ')
    roe   = [float(x.replace('%','').replace(',','')) for x in stock[stock.detail == 'ROE'].values[0][3:] if str(x).strip() not in symbol]
    nY    = len(roe)
    try:
        if 'REITS' in stock.category.unique():
            pass;
        else:
            for y in [float(x.replace('%','').replace(',','')) for x in stock[stock.detail == 'NP'].values[0][3:3+nY] if str(x).strip() not in symbol]: assert(y >= 0)
#         try:
#             if 'REITS' in stock.category.unique():
#                 pass;
#             else:
#                 for y in [float(x.replace('%','').replace(',','')) for x in stock[stock.detail == 'EPS GR'].values[0][3:].tolist() if x not in symbol]: assert(y >= 0)  
        try:
            if 'REITS' in stock.category.unique():
                pass;
            else:
                assert(nY >= 5)
            try:
                roe_avg       = np.around(np.mean(roe), 2)
                roe_std       = np.around(np.std(roe), 2)
                eps_gr        = [float(x.replace('%','').replace(',','')) if str(x).strip() not in symbol else 0.0 for x in stock[stock.detail == 'EPS GR'].values[0][3:3+nY]]
                eps_gr_avg    = np.around(sum(eps_gr)/nY, 2)
                eps_gr_std    = np.around(np.sqrt(np.mean([np.power(eps_gr_avg - x, 2) for x in eps_gr])), 2)
                np_margin     = [float(x.replace('%','').replace(',','')) if str(x).strip() not in symbol else 0.0 for x in stock[stock.detail == 'NP Margin'].values[0][3:]]
                np_margin_avg = np.around(np.mean(np_margin), 2)
                np_margin_std = np.around(np.std(np_margin), 2)
                pe            = [float(x.replace(',','')) if str(x).strip() not in symbol else 0.0 for x in stock[stock.detail == 'P/E'].values[0][3:3+nY]]
                pe_now        = pe[0]
                pe_mid        = np.around((max(pe) + min(pe))/2, 2)
                pe_avg        = np.around(np.mean(pe), 2)
                pe_std        = np.around(np.std(pe), 2)
                eps           = [float(x.replace(',','')) if str(x).strip() not in symbol else 0.0 for x in stock[stock.detail == 'EPS'].values[0][3:3+nY]]
                eps_now       = eps[0]
                eps_avg       = np.around(np.mean(eps), 2)
                eps_std       = np.around(np.std(eps), 2)
                dps           = [float(x.replace(',','')) if str(x).strip() not in symbol else 0.0 for x in stock[stock.detail == 'DPS'].values[0][3:3+nY]]
                dps_now       = dps[0]
                dps_avg       = np.around(np.mean(dps), 2)
                dps_std       = np.around(np.std(dps), 2)
                dpayout_avg   = np.around(np.mean([dps[x] / eps[x] * 100 for x in range(nY) if eps[x] > 0.0]),2)#if (dps[x] > 0.0) & (eps[x] > 0.0)
                dpayout_std   = np.around(np.std([dps[x] / eps[x] * 100 for x in range(nY) if eps[x] > 0.0]),2)#if (dps[x] > 0.0) & (eps[x] > 0.0)
                dy            = [float(x.replace(',','')) if str(x).strip() not in symbol else 0.0 for x in stock[stock.detail == 'DY'].values[0][3:3+nY]]
                dy_now        = dy[0]
                dy_avg        = np.around(np.mean(dy),2)
                dy_std        = np.around(np.std(dy),2)

                net_profit = [int(x.replace(',','')) if str(x).strip() not in symbol else 0 for x in stock[stock.detail == 'NP'].values[0][3:3+nY] ]
                nosh       = [int(x.replace(',','')) if str(x).strip() not in symbol else 0 for x in stock[stock.detail == 'NOSH'].values[0][3:3+nY]]
                price      = [float(x.replace(',','')) if str(x).strip() not in symbol else 0.0 for x in stock[stock.detail == 'Price'].values[0][3:3+nY]]
                temp       = [np.around(net_profit[x] / (nosh[x] * price[x]) * 100, 2) for x in range(len(net_profit)) if price[x] != 0]
                roi_avg    = np.around(np.mean(temp), 2)
                roi_std    = np.around(np.std(temp), 2)
                roi        = np.around(temp[0], 2)
                lt_price   = price[0]

                nMerge    = 0
                nSplit    = 0
                dates     = [datetime.strptime(x, '%d/%m/%y') for x in stock[stock.detail == 'Year End'].values[0][3:3+nY] if str(x).strip() not in symbol]
                last_date = dates[-1]
                annual    = dates[0]
                temp1     = data2[data2.code == i]   
#                 temp1     = temp1[temp1.split_date.apply(lambda x: datetime.strptime(x, '%m/%d/%Y')) >= last_date]
                temp1     = temp1[temp1.split_date >= last_date]
                for n, row in temp1.iterrows():
                    split_ratio = row.ratio.split(':')
                    if len(split_ratio) == 2:
                        if split_ratio[0] < split_ratio[-1]: nMerge += 1   
                        else: nSplit += 1
                    else:
                        error4a.append([str(i).zfill(4), 'Split date format error'])
                result4.append([str(i).zfill(4), nY, roi, roi_avg, roi_std, roe_avg, roe_std, eps_gr_avg, eps_gr_std, np_margin_avg, 
                                np_margin_std, pe_now, pe_mid, pe_avg, pe_std, eps_now, eps_avg, eps_std, dps_now, dps_avg, 
                                dps_std, dpayout_avg, dpayout_std, dy_avg, dy_std, lt_price, nMerge, nSplit, annual])
            except:
                error4.append([str(i).zfill(4), 'Calculation Error'])
        except:
            error4.append([str(i).zfill(4), 'Less than 5 years'])
#         except:
#             error3.append([str(i).zfill(4), 'Negavtive EPS Growth Rate'])
    except:
        error4.append([str(i).zfill(4), 'Negative Net Profit'])
        
er_df_4 = pd.DataFrame(error4, columns=['code','reason'])
display(er_df_4.reason.value_counts())
if len(error4a) != 0: print(len(error4a))

data4 = pd.DataFrame(result4, columns=['code','nYear','ROI','ROI avg','ROI std','ROE avg','ROE std','EPS GR avg','EPS GR std',
                                       'NPM avg','NPM std','P/E','P/E mid','P/E avg','P/E std','EPS','EPS avg','EPS std',
                                       'DPS','DPS avg','DPS std','DPayout avg','DPayout std','DY avg','DY std','price',
                                       'nMerge','nSplit','Annual'])
# data4 = data4[data4['ROI avg'] >= 6]
# data4 = data4[data4['ROE avg'] >= 6]
# data4 = data4[data4['EPS GR avg'] >= 6]
# data4 = data4[data4['P/E'] < data4['P/E mid']]
data4 = data4.merge(data1,'left')
data4 = data4.merge(stock_code,'left')
print(data4.shape)
data4.sort_values('ROI',ascending=False,inplace=True)
data4.reset_index(drop=True,inplace=True)
data4.to_csv(out_dir+'klse_stock_stage_4.csv',index=False)
display(data4.head(1))

100%|████████████████████████████████████████████████████████████████████████████| 787/787 [00:03<00:00, 198.94it/s]


Negative Net Profit    443
Calculation Error       95
Less than 5 years       15
Name: reason, dtype: int64

(234, 36)


,code,nYear,ROI,ROI avg,ROI std,ROE avg,ROE std,EPS GR avg,EPS GR std,NPM avg,...,nMerge,nSplit,Annual,category,current_price,52w low,52w high,market cap,NTA,name
0,5105,10,130.19,28.98,35.37,17.31,12.73,182.14,440.18,11.96,...,0,0,2019-12-31,Industrial Products & Services,1.93,1.6,3.99,370.9M,9.019,CAN-ONE BERHAD


In [10]:
data4 = pd.DataFrame(result4, columns=['code','nYear','ROI','ROI avg','ROI std','ROE avg','ROE std','EPS GR avg','EPS GR std',
                                       'NPM avg','NPM std','P/E','P/E mid','P/E avg','P/E std','EPS','EPS avg','EPS std',
                                       'DPS','DPS avg','DPS std','DPayout avg','DPayout std','DY avg','DY std','price',
                                       'nMerge','nSplit','Annual'])
# data4 = data4[data4['ROI avg'] >= 6]
# data4 = data4[data4['ROE avg'] >= 6]
# data4 = data4[data4['EPS GR avg'] >= 6]
# data4 = data4[data4['P/E'] < data4['P/E mid']]
data4 = data4.merge(data1,'left')
data4 = data4.merge(stock_code,'left')
print(data4.shape)
data4.sort_values('ROI',ascending=False,inplace=True)
data4.reset_index(drop=True,inplace=True)
data4.to_csv(out_dir+'klse_stock_stage_4.csv',index=False)
display(data4.head(1))

(234, 36)


,code,nYear,ROI,ROI avg,ROI std,ROE avg,ROE std,EPS GR avg,EPS GR std,NPM avg,...,nMerge,nSplit,Annual,category,current_price,52w low,52w high,market cap,NTA,name
0,5105,10,130.19,28.98,35.37,17.31,12.73,182.14,440.18,11.96,...,0,0,2019-12-31,Industrial Products & Services,1.93,1.6,3.99,370.9M,9.019,CAN-ONE BERHAD


In [9]:
data4[data4.price > data4.NTA]

,code,nYear,ROI,ROI avg,ROI std,ROE avg,ROE std,EPS GR avg,EPS GR std,NPM avg,...,nMerge,nSplit,Annual,category,current_price,52w low,52w high,market cap,NTA,name
13,7252,10,15.52,11.72,7.27,14.81,8.93,352.47,860.46,7.04,...,0,0,2019-12-31,Consumer Products & Services,0.850,0.850,1.430,255.0M,1.0400,TEO SENG CAPITAL BERHAD
16,5147,10,13.16,10.79,4.20,12.39,5.38,15.24,89.07,2.29,...,0,0,2019-12-31,Industrial Products & Services,0.455,0.450,0.765,123.8M,0.5700,SAMCHEM HOLDINGS BERHAD
20,7029,10,11.49,12.95,4.93,6.65,3.41,39.89,99.93,4.65,...,0,0,2019-12-31,Industrial Products & Services,0.990,0.695,3.200,54.1M,1.9400,MASTER-PACK GROUP BERHAD
21,5100,10,10.97,9.68,3.25,9.66,2.47,8.77,43.93,5.68,...,0,0,2019-12-31,Industrial Products & Services,0.930,0.920,1.130,174.5M,1.0200,BP PLASTICS HOLDING BHD
25,6645,10,10.62,6.68,1.64,26.35,3.19,11.41,18.60,36.10,...,0,0,2019-03-31,Transportation & Logistics,4.350,4.130,5.000,"2,311.3M",2.0700,LINGKARAN TRANS KOTA HOLDINGS BERHAD
26,5258,10,10.14,9.56,2.44,14.32,1.94,8.15,13.40,18.93,...,0,0,2019-12-31,Financial Services,3.440,3.440,4.750,"6,166.8M",3.3400,BIMB HOLDINGS BERHAD
29,3301,10,9.23,10.78,3.88,15.63,4.33,18.84,73.01,12.47,...,0,0,2019-06-30,Consumer Products & Services,7.970,7.970,11.600,"2,613.4M",5.4800,HONG LEONG INDUSTRIES BERHAD
32,2488,10,8.49,7.97,0.57,11.51,1.70,6.53,12.73,35.31,...,0,0,2019-03-31,Financial Services,1.870,1.840,4.220,"2,895.0M",3.8200,ALLIANCE BANK MALAYSIA BERHAD
34,1155,10,8.44,7.49,1.39,11.64,2.35,12.84,34.43,18.53,...,0,0,2019-12-31,Financial Services,7.880,7.850,9.550,"88,581.9M",7.2563,MALAYAN BANKING BERHAD
37,7205,10,8.24,6.51,1.79,12.46,3.14,10.00,20.19,11.24,...,0,0,2019-12-31,Consumer Products & Services,1.700,1.680,2.220,389.0M,1.0900,COCOALAND HOLDINGS BERHAD


### Stage 5: Intrinsic Value & Fair Value

In [10]:
result5 = []
for n, x in tqdm(data4.iterrows(), total=len(data4)):
    price_10y = [np.around(x['price'] * ((1 + (x['EPS GR avg'] / 100))**i) ,2) for i in range(5)]
    total_eps = np.around(sum(price_10y), 2)
    expected_price = np.around(price_10y[-1] * x['P/E mid'],2)
    total_dvd = np.around(total_eps * (x['DPayout avg'] / 100), 2)
    total_return = np.around(total_eps + total_dvd, 2)
    intrinsic = np.around(total_return / ((1 + (x['EPS GR avg'] / 100))**5), 2)
    price_A = np.around(intrinsic * 0.75, 2)
    price_B = np.around((x['52w high'] - x['52w low']) * 0.33 + x['52w low'], 2)
    result5.append([str(x['code']), intrinsic, price_A, price_B])

data5 = data4.merge(pd.DataFrame(result5, columns=['code','intrinsic','price_A','price_B']))
print(data5.shape)
data5.to_csv(out_dir+'klse_stock_intrinsic.csv',index=False)
data5.head(1)

100%|█████████████████████████████████████████████████████████████████████████████| 51/51 [00:00<00:00, 3186.69it/s]


(51, 39)


,code,nYear,ROI,ROI avg,ROI std,ROE avg,ROE std,EPS GR avg,EPS GR std,NPM avg,...,category,current_price,52w low,52w high,market cap,NTA,name,intrinsic,price_A,price_B
0,5105,10,130.19,28.98,35.37,17.31,12.73,182.14,440.18,11.96,...,Industrial Products & Services,2.5,2.4,3.99,480.4M,9.019,CAN-ONE BERHAD,1.74,1.3,2.92


In [11]:
temp = data5.columns.tolist()
data5 = data5[[temp[-4],temp[29],temp[28]]+temp[26:28]+[temp[-5]]+temp[-9:-5]+temp[-3:]+
              temp[:2]+[temp[25]]+temp[2:9]+temp[15:18]+temp[9:15]+temp[18:25]]
data5.head(1)

,name,category,Annual,nMerge,nSplit,NTA,current_price,52w low,52w high,market cap,...,P/E mid,P/E avg,P/E std,DPS,DPS avg,DPS std,DPayout avg,DPayout std,DY avg,DY std
0,CAN-ONE BERHAD,Industrial Products & Services,2019-12-31,0,0,9.019,2.5,2.4,3.99,480.4M,...,6.05,6.42,3.07,6.0,4.2,0.87,11.22,5.91,1.88,0.62


In [12]:
data5.to_csv(out_dir+'klse_stock_intrinsic.csv',index=False)

In [13]:
# Stocks which have prices lower than fair value price B
data5[data5.price < data5.price_B]

,name,category,Annual,nMerge,nSplit,NTA,current_price,52w low,52w high,market cap,...,P/E mid,P/E avg,P/E std,DPS,DPS avg,DPS std,DPayout avg,DPayout std,DY avg,DY std
0,CAN-ONE BERHAD,Industrial Products & Services,2019-12-31,0,0,9.019,2.50,2.400,3.990,480.4M,...,6.05,6.42,3.07,6.0,4.20,0.87,11.22,5.91,1.88,0.62
1,SIG GASES BERHAD,Industrial Products & Services,2019-12-31,0,0,0.550,0.41,0.400,0.995,76.9M,...,24.40,24.30,14.54,105.7,11.22,31.49,50.58,67.48,27.17,77.94
10,ORNAPAPER BERHAD,Industrial Products & Services,2019-12-31,0,0,2.360,0.96,0.875,1.420,72.2M,...,8.29,7.36,1.41,3.0,1.35,1.36,10.34,11.21,1.32,1.36
19,MKH BERHAD,Property,2019-09-30,0,0,2.840,1.13,1.100,1.860,662.8M,...,10.40,9.26,2.27,3.5,5.55,2.57,29.22,13.28,2.99,1.26
25,LINGKARAN TRANS KOTA HOLDINGS BERHAD,Transportation & Logistics,2019-03-31,0,0,2.070,4.35,4.130,5.000,"2,311.3M",...,17.06,15.86,3.82,25.0,20.50,3.77,74.46,16.22,4.76,0.64
38,LEE SWEE KIAT GROUP BERHAD,Consumer Products & Services,2019-12-31,0,0,0.350,0.50,0.450,0.950,83.9M,...,12.28,11.03,3.94,2.5,0.75,1.01,16.29,20.36,1.49,1.85
41,SCICOM (MSC) BERHAD,Industrial Products & Services,2019-06-30,0,0,0.290,0.75,0.600,1.300,266.6M,...,14.86,14.99,4.84,5.5,5.65,2.87,73.61,18.63,5.13,1.05
43,PINTARAS JAYA BHD,Construction,2019-06-30,0,0,1.980,2.73,2.050,3.430,452.8M,...,18.27,14.34,8.99,12.0,18.40,3.44,84.91,61.06,6.11,1.71
47,POWER ROOT BERHAD,Consumer Products & Services,2019-03-31,0,0,0.660,1.89,1.260,2.450,773.6M,...,24.16,16.13,12.39,8.0,7.72,3.40,125.93,86.07,6.31,3.57


In [14]:
# Stocks which have prices lower than fair value price B and average NPM of 10% or higher
data5[(data5.price < data5.price_B) & (data5['NPM avg'] >= 10)]

,name,category,Annual,nMerge,nSplit,NTA,current_price,52w low,52w high,market cap,...,P/E mid,P/E avg,P/E std,DPS,DPS avg,DPS std,DPayout avg,DPayout std,DY avg,DY std
0,CAN-ONE BERHAD,Industrial Products & Services,2019-12-31,0,0,9.019,2.50,2.40,3.99,480.4M,...,6.05,6.42,3.07,6.0,4.20,0.87,11.22,5.91,1.88,0.62
19,MKH BERHAD,Property,2019-09-30,0,0,2.840,1.13,1.10,1.86,662.8M,...,10.40,9.26,2.27,3.5,5.55,2.57,29.22,13.28,2.99,1.26
25,LINGKARAN TRANS KOTA HOLDINGS BERHAD,Transportation & Logistics,2019-03-31,0,0,2.070,4.35,4.13,5.00,"2,311.3M",...,17.06,15.86,3.82,25.0,20.50,3.77,74.46,16.22,4.76,0.64
41,SCICOM (MSC) BERHAD,Industrial Products & Services,2019-06-30,0,0,0.290,0.75,0.60,1.30,266.6M,...,14.86,14.99,4.84,5.5,5.65,2.87,73.61,18.63,5.13,1.05
43,PINTARAS JAYA BHD,Construction,2019-06-30,0,0,1.980,2.73,2.05,3.43,452.8M,...,18.27,14.34,8.99,12.0,18.40,3.44,84.91,61.06,6.11,1.71


In practical, intrinsic value and price A are way beyond from the current price because there is no consistency of positive growth rate for any stock, my suggestion will be excluding the idea of positive growth rate, but filter stocks with positive earnings or net profit margin at 5% or 10%  
 <br />
Another method would be filtering stocks according to the dividend yield over the past 10 years. Although this method is not recommended by the author,but it returns more stock options and most of them has dividend yield of 10% or higher

### Process by Dividend Yield

In [15]:
data6 = pd.DataFrame(result4, columns=['code','nYear','ROI','ROI avg','ROI std','ROE avg','ROE std','EPS GR avg','EPS GR std',
                                       'NPM avg','NPM std','P/E','P/E mid','P/E avg','P/E std','EPS','EPS avg','EPS std',
                                       'DPS','DPS avg','DPS std','DPayout avg','DPayout std','DY avg','DY std','price',
                                       'nMerge','nSplit','Annual'])
data6 = data6.merge(data1,'left')
data6 = data6.merge(stock_code,'left')
print(data6.shape)
data6.sort_values('DY avg',ascending=False,inplace=True)
data6.reset_index(drop=True,inplace=True)
data6.to_csv(out_dir+'klse_stock_dividend_yield.csv',index=False)
data6.head(1)

(242, 36)


,code,nYear,ROI,ROI avg,ROI std,ROE avg,ROE std,EPS GR avg,EPS GR std,NPM avg,...,nMerge,nSplit,Annual,category,current_price,52w low,52w high,market cap,NTA,name
0,5181,10,105.25,15.02,30.21,12.23,21.86,165.15,452.86,6.87,...,0,0,2019-12-31,Industrial Products & Services,0.41,0.4,0.995,76.9M,0.55,SIG GASES BERHAD


In [16]:
temp = data6.columns.tolist()
data6 = data6[[temp[-1],temp[29],temp[28]]+temp[26:28]+[temp[-2]]+temp[-6:-2]+
              temp[:2]+[temp[25]]+temp[2:9]+temp[15:18]+temp[9:15]+temp[18:25]]
data6.head(1)

,name,category,Annual,nMerge,nSplit,NTA,current_price,52w low,52w high,market cap,...,P/E mid,P/E avg,P/E std,DPS,DPS avg,DPS std,DPayout avg,DPayout std,DY avg,DY std
0,SIG GASES BERHAD,Industrial Products & Services,2019-12-31,0,0,0.55,0.41,0.4,0.995,76.9M,...,24.4,24.3,14.54,105.7,11.22,31.49,50.58,67.48,27.17,77.94


In [17]:
data6.to_csv(out_dir+'klse_stock_dividend_yield.csv',index=False)

In [18]:
data6.sort_values('DY avg',ascending=False)

,name,category,Annual,nMerge,nSplit,NTA,current_price,52w low,52w high,market cap,...,P/E mid,P/E avg,P/E std,DPS,DPS avg,DPS std,DPayout avg,DPayout std,DY avg,DY std
0,SIG GASES BERHAD,Industrial Products & Services,2019-12-31,0,0,0.5500,0.410,0.400,0.995,76.9M,...,24.40,24.30,14.54,105.70,11.22,31.49,50.58,67.48,27.17,77.94
1,STAR MEDIA GROUP BERHAD,Telecommunications & Media,2019-12-31,0,0,1.1100,0.295,0.295,0.810,217.9M,...,52.60,26.23,27.71,2.00,22.41,19.49,186.59,123.47,9.87,7.04
2,PERUSAHAAN SADUR TIMAH MALAYSIA (PERSTIMA) BHD,Industrial Products & Services,2019-03-31,0,0,3.9600,3.540,3.530,5.000,351.5M,...,10.29,10.70,2.80,10.00,35.35,9.02,85.51,35.91,8.22,2.90
3,CLASSIC SCENIC BERHAD,Consumer Products & Services,2019-12-31,0,0,0.7700,0.755,0.750,1.140,91.0M,...,13.18,13.72,3.78,5.00,8.56,2.22,99.04,13.49,7.80,2.41
4,AMANAHRAYA REAL ESTATE INVESTMENT TRUST,Real Estate Investment Trusts,2019-12-31,0,0,1.3378,0.680,0.655,0.865,389.8M,...,14.04,10.63,5.27,6.20,6.58,0.65,77.60,38.48,7.46,0.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,MALAYSIA AIRPORTS HOLDINGS BERHAD,Transportation & Logistics,2019-12-31,0,0,5.6204,5.670,5.420,8.880,"9,407.6M",...,116.00,56.33,65.63,0.00,0.00,0.00,0.00,0.00,0.00,0.00
238,TA GLOBAL BERHAD,Property,2019-12-31,0,0,0.5900,0.230,0.225,0.280,"1,224.0M",...,16.47,12.91,8.52,0.00,0.00,0.00,0.00,0.00,0.00,0.00
239,TA ENTERPRISE BERHAD,Financial Services,2019-12-31,0,0,1.4800,0.500,0.490,0.690,856.0M,...,8.41,9.28,4.72,0.01,0.00,0.00,0.01,0.04,0.00,0.01
240,ABM FUJIYA BERHAD,Industrial Products & Services,2019-12-31,0,0,0.8900,0.490,0.370,0.710,88.2M,...,24.27,20.58,12.80,0.00,0.00,0.00,0.00,0.00,0.00,0.00


 #### Thank you for reading, using, and exploring this, hope to see you soon